In [58]:
# Script to fix change a bunch of SCORM settings by Andy Hicken, 10 Dec 2019. 
# I used this from the Anaconda distribution of Python. 
# Selenium for Python has to be installed before this, and you'll need the geckodriver driver for Firefox.

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# used for explicit wait
from selenium.webdriver.common.by import By
# allows "implicit wait" (timer)
from selenium.webdriver.support.ui import WebDriverWait
# allows "explicit wait" (wait until you see a certain thing)
from selenium.webdriver.support import expected_conditions as EC

import time 
import sys
import io

# Update this: 
# this sets USERNAME, PASSWORD, login_URL, and site_URL from a private file (update this to your file)
%run \\jupiter\users\ahicken\app_keys\scorm_keys.py

In [59]:
# login

# use Chrome for the scraping 
# driver = webdriver.Chrome()

# use Firefox for the scraping
driver = webdriver.Firefox()
# wait 2 seconds for Javascript to load 
driver.implicitly_wait(2000)
# NOTE: "Once set, the implicit wait is set for the life of the WebDriver object."

driver.get(login_URL)

# log in 
# username
username = driver.find_element_by_id("username")
username.clear()
username.send_keys(USERNAME)

# password
password = driver.find_element_by_id("password")
password.clear()
password.send_keys(PASSWORD)

# click
driver.find_element_by_id("loginbtn").click()
 

In [34]:
# function: check if the SCORM is set as a requirement in the course completion settings.
def checkCourseSettings(course_id, scorm_id): 
    # go to the course completion settings
    completion_URL = site_URL + '/course/completion.php?id=' + str(course_id)
    driver.get(completion_URL)
    
    # find the SCORM links and return "multiple SCORMs" if appopriate
    # might not do this - only necessary if SCORMs are required for completion
    
    # note: the ID for the checkbox includes the activity ID, in the form 'id_criteria_activity_1032'
    # this code returns true if the checkbox is selected, and false if it is not:    
    return driver.find_element_by_id("id_criteria_activity_" + str(scorm_id)).is_selected()
    


In [35]:
# function: reset completion for a SCORM module
def resetCompletion(course_id, scorm_id): 
    # go to the SCORM settings
    SCORM_settings_URL = site_URL + '/course/modedit.php?update=' + str(scorm_id)
    driver.get(SCORM_settings_URL)

    
    # expand all headers if necessary
    try:
        driver.find_element_by_link_text('Expand all').click()
    except:
        pass
    
    # check if the Require View setting is checked. Return a message if not. 
    if not driver.find_element_by_id("id_completionview").is_selected():
        return "require view was not checked for " + str((course_id, scorm_id))
    
    # try to click the Reset Completion button. 
    # uncheck the checkbox, save and return a message if it not present.
    try: 
        driver.find_element_by_id('id_unlockcompletion').click()
    except:
        driver.find_element_by_id("id_completionview").click() # Require View checkbox
        driver.find_element_by_id("id_submitbutton2").click() # Save and return to course
        return "reset. activity completion was not locked for " + str((course_id, scorm_id))
    
    # uncheck the Require View setting, then save. 
    driver.find_element_by_id("id_completionview").click() # Require View checkbox
    driver.find_element_by_id("id_submitbutton2").click() # Save and return to course
    return "success for " + str((course_id, scorm_id))
    
    
    

In [36]:
# function: main loop
def resetCompletionLoop(list_of_ids):
    for duple in list_of_ids:
        if checkCourseSettings(duple[0], duple[1]) == False: 
            # (we get here if the completion checkbox for the SCORM is NOT checked)
            response = resetCompletion(duple[0], duple[1])
            print(response)
        else:
            print("completion depended on " + str(duple))
            

In [56]:
# list of tuples with (course ID, SCORM ID)
test_list = [
    (454,2631),
    (455,2635),
    (312,5388),
    (306,5390),
    (962,5438),
    (337,5444),
    (973,5500)
]

In [57]:
# call 
resetCompletionLoop(test_list)

success for (454, 2631)
success for (455, 2635)
success for (312, 5388)
success for (306, 5390)
success for (962, 5438)
success for (337, 5444)
success for (973, 5500)
